# Will a Viral Song Enter the Top 200 Streamed Tracks?

In [128]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import types
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

from connections import password

# Data Import

In [129]:
# Establish SQL connection
connection_string = (f"root:{password}@localhost/spot_db")
engine = create_engine(f"mysql://{connection_string}")# , pool_recycle=3600, pool_pre_ping=True)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save reference to the table
_features = Base.classes.features
_top = Base.classes.top_200_daily
# Create connection object
session = Session(engine)

In [130]:
features = pd.read_sql(session.query(_features).statement, session.bind)

In [131]:
top = pd.read_sql(session.query(_top).statement, session.bind)

# Data Pre-Processing

In [132]:
features['label'] = features.ID.isin(top.ID)
features.head()

,index,Acousticness,Danceability,Duration_ms,Energy,ID,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time_Signature,Valence,label
0,1560,0.13100,0.748,188491,0.627,000xQL6tZNLJzIrtIgxqSl,0.00000,7,0.0852,-6.029,1,0.0644,120.963,4,0.524,True
1,6494,0.08220,0.353,193680,0.755,003eoIwxETJujVWmNFMoZy,0.00000,1,0.3900,-6.276,0,0.7330,191.153,4,0.437,True
2,1345,0.00239,0.588,262347,0.885,004S8bMhFQjnbuqvdh6W71,0.00246,9,0.0862,-6.267,0,0.0654,149.031,4,0.928,False
3,7841,0.00304,0.619,218747,0.762,007d7JT41sSc1HqWTs4uw7,0.00000,0,0.1220,-6.738,0,0.0692,122.916,4,0.324,True
4,5989,0.28700,0.620,284856,0.625,00B7TZ0Xawar6NZ00JFomN,0.00000,9,0.3140,-7.438,1,0.5530,167.911,4,0.665,True


In [133]:
features.tail()

,index,Acousticness,Danceability,Duration_ms,Energy,ID,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time_Signature,Valence,label
11715,8088,0.00898,0.612,188253,0.799,7zVCrzzEJU7u24sbJPXA5W,0.000000,10,0.1720,-4.603,1,0.0275,125.976,4,0.687,True
11716,268,0.07950,0.717,196173,0.587,7zvKFw17XyoBUx9mHiwzPy,0.000019,7,0.1920,-8.970,1,0.3680,112.440,5,0.336,False
11717,972,0.63500,0.415,276875,0.415,7zxrA6P9OWCc1nXF6GGLIp,0.005210,10,0.0874,-10.784,0,0.0501,139.822,5,0.179,False
11718,1072,0.10500,0.748,179773,0.723,7zxRMhXxJMQCeDDg0rKAVo,0.000000,0,0.0949,-6.213,1,0.3470,154.966,4,0.350,True
11719,4659,0.06920,0.863,246009,0.609,7zyHb6wrRALMIyCZgwLw5u,0.000000,10,0.1410,-3.556,1,0.2700,115.054,4,0.701,False


In [134]:
X = features.drop(['label', 'index', 'ID'], axis = 1)
y = features['label']

In [135]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

In [136]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 1, stratify = y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode dataset
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Convert encoded labels to one-hot-encodeing


y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create Deep Learning Model

In [137]:
from keras.models import Sequential
from keras.layers import Dense

### Viral and Top Model

In [151]:
# # Create model and add layers
# model = Sequential()
# model.add(Dense(units=5, activation='relu', input_dim=13))
# # model.add(Dense(units=4, activation='relu'))
# model.add(Dense(units=2, activation='softmax'))

In [174]:
X_train_scaled[0]

array([-0.10316787,  1.34263848, -1.26436601, -0.27601791, -0.30195312,
        0.20132525, -0.30861145,  0.23329864, -1.25546869,  0.36569025,
       -0.06113256,  0.15080498, -0.07672952])

### Genre and Top Model

In [167]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=20, activation='relu', input_dim=13))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [168]:
# Compile and fit the model
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(X_train_scaled, 
         y_train_categorical,
         epochs = 20,
         shuffle = True,
         verbose = 2)

Epoch 1/20
 - 1s - loss: 0.7232 - acc: 0.5625
Epoch 2/20
 - 0s - loss: 0.5925 - acc: 0.6861
Epoch 3/20
 - 0s - loss: 0.5815 - acc: 0.6966
Epoch 4/20
 - 0s - loss: 0.5762 - acc: 0.6997
Epoch 5/20
 - 0s - loss: 0.5718 - acc: 0.7019
Epoch 6/20
 - 0s - loss: 0.5687 - acc: 0.7025
Epoch 7/20
 - 0s - loss: 0.5658 - acc: 0.7058
Epoch 8/20
 - 0s - loss: 0.5646 - acc: 0.7064
Epoch 9/20
 - 0s - loss: 0.5623 - acc: 0.7044
Epoch 10/20
 - 0s - loss: 0.5611 - acc: 0.7068
Epoch 11/20
 - 0s - loss: 0.5596 - acc: 0.7081
Epoch 12/20
 - 0s - loss: 0.5585 - acc: 0.7082
Epoch 13/20
 - 0s - loss: 0.5579 - acc: 0.7075
Epoch 14/20
 - 0s - loss: 0.5570 - acc: 0.7085
Epoch 15/20
 - 0s - loss: 0.5556 - acc: 0.7104
Epoch 16/20
 - 0s - loss: 0.5558 - acc: 0.7073
Epoch 17/20
 - 0s - loss: 0.5538 - acc: 0.7126
Epoch 18/20
 - 0s - loss: 0.5533 - acc: 0.7097
Epoch 19/20
 - 0s - loss: 0.5522 - acc: 0.7121
Epoch 20/20
 - 0s - loss: 0.5517 - acc: 0.7131


# Performance

In [169]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.564079308102969, Accuracy: 0.7112627985534408


# Prediction Test

In [170]:
encoded_predictions = model.predict_classes(X_test_scaled[840:845])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels:     {list(y_test[840:845])}")

Predicted classes: [False False False  True False]
Actual Labels:     [False, True, True, True, False]


C:\Users\Russell Bogen\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [171]:
# Save the model
model.save("Feature_Model_Trained.h5")